# **2.11_series_extended**

### 一、Series的运算

#### (一)、Series和Series之间

在Series和Series之间,可以做加减乘除等各种运算，Pandas会自动根据索引去排序并对齐

1. 如果某个索引只在其中一个Series出现的话，结果就会是NaN,表示not a number，说明无法得到计算值

    也就是说，由于Series之间的计算会自动进行索引对齐，只有当某个索引同时出现在两个Series里时，结果里才会有对应的值

    ***按照什么进行排序? 数字按照大小，英文按照字母顺序，数字英文中文可以混在一起排序对齐，数字在英文前，英文在中文前。中文排序方式或许是ASCII***

In [1]:
import pandas as pd
s1 = pd.Series([1, 4, 2, 3, 5], index=[1, 3, 5, 7, 9])
s2 = pd.Series([8, 1, 7 ,3 ,9], index=[1, 2, 3, 5, 10])
s1 + s2

1      9.0
2      NaN
3     11.0
5      5.0
7      NaN
9      NaN
10     NaN
dtype: float64

In [2]:
s0 = pd.Series({1: 1, "只": 1, "住": 1, "num": 1, 3: 1})
s0 + s1

1      2.0
3      5.0
5      NaN
7      NaN
9      NaN
num    NaN
住      NaN
只      NaN
dtype: float64

2. 如果你希望给缺失的值一个默认值的话，可以用方法而不是运算符号进行运算，然后给fill_value这个参数传入一个值。

   用符号的话我们没法额外传参，但用方法的话就可以

In [3]:
s1.add(s2, fill_value=0)

1      9.0
2      1.0
3     11.0
5      5.0
7      3.0
9      5.0
10     9.0
dtype: float64

    等同于s1 + s2,并同时给两边缺失的值一个默认值0

In [4]:
s1.sub(s2, fill_value=0)

1    -7.0
2    -1.0
3    -3.0
5    -1.0
7     3.0
9     5.0
10   -9.0
dtype: float64

In [5]:
s1.mul(s2, fill_value=0)

1      8.0
2      0.0
3     28.0
5      6.0
7      0.0
9      0.0
10     0.0
dtype: float64

In [6]:
s1.div(s2, fill_value=0)

1     0.125000
2     0.000000
3     0.571429
5     0.666667
7          inf
9          inf
10    0.000000
dtype: float64

3. 优势

   Series之间的操作会根据索引自动对齐的好处是，由于一般我们会利用标签索引表示不同对象的数据,那即使不同Series里数据顺序不一样，计算时也会根据索引自动对齐

#### (二)、聚合运算

1. NumPy数组的统计方法，包括max, min, sum, mean，Pandas的Series对象也有相同名字的方法

In [7]:
print(s1.max())
print(s1.min())
print(s1.sum())
print(s1.mean())

5
1
15
3.0


2. describe方法，是Series特有的一个强大的方法，describe方法能直接告诉我们很多这个Series的统计信息，

   包括：元素个数、平均数、标准差、最小值、四分位数、最大值

In [8]:
s1.describe()

count    5.000000
mean     3.000000
std      1.581139
min      1.000000
25%      2.000000
50%      3.000000
75%      4.000000
max      5.000000
dtype: float64

#### (三)、Series和单个数字之间

与NumPy数组的广播机制一样，在Pandas Series里，单个数字和Series之间进行操作的时候，操作会被自动运用到Series里每个元素上

In [9]:
s1 * 3

1     3
3    12
5     6
7     9
9    15
dtype: int64

### 二、对元素分别执行相同操作

1. apply方法，接收函数作为参数，然后调用时把Series里各个元素，分别作为那个函数的参数，返回的Series里的元素，就是那个函数对原始Series里各个元素调用后的结果

    **apply方法不改变原始Series，而是会返回一个新的Series**

***apply相当于是高阶函数***
        
***注意传入的函数后面不要跟括号，因为不是要把函数调用后的结果，去作为apply的参数，而是把函数本身给apply***
    
***apply的定义语句里肯定包括：***
 
***a.让每一个Series里面的元素作为参数，调用函数***
    
***b.将每一个元素调用函数得到的结果，组成的新Series。***

    优势:apply方法大大增加了我们操作Series的灵活性，能定义出来的函数，我们都可以作用在Series的各个元素上，帮我们得到新的Series

    应用场景：当前有5名学生的成绩所组成的Series,索引为学生名字，我们希望能得到每个成绩对应的等级：90及以上是A，80到90是B，70到80是C，70以下是D。我们知道怎么根据分数数字得到对应等级，只需要get_grade_from_score函数即可。现在问题在于，如何对Series里每个元素，都运用这个函数，得到对应结果组成的新Series。新方法，apply方法可以实现这一步。

In [10]:
scores = pd.Series({"小明": 92, "小红": 67, "小杰": 70, "小丽": 88, "小华": 76})
def get_grade_from_score(score):
    if score >= 90:
        return "A"
    elif score >= 80:
        return "B"
    elif score >= 70:
        return "C"
    else:
        return "D"


grades = scores.apply(get_grade_from_score)
grades

小明    A
小红    D
小杰    C
小丽    B
小华    C
dtype: object

2. 除了传入定义好的函数名，在函数逻辑比较简单的时候，匿名函数也可以应用在这里

In [11]:
half_scores = scores.apply(lambda x: 0.5*x)
half_scores

小明    46.0
小红    33.5
小杰    35.0
小丽    44.0
小华    38.0
dtype: float64

### 三、转换数据类型 

astype方法：转换Series的数据类型

In [15]:
scores = scores.astype(str)
scores

小明    92
小红    67
小杰    70
小丽    88
小华    76
dtype: object

### 四、针对字符串Series，保留Series每个元素的某一部分

str.slice方法

str是Series类自带的一个属性，会返回一个包含了很多字符串相关操作方法的,StringMethods类的实例(返回实例才可以调用方法)，对这个StringMethods实例调用slice方法，就会分别保留Series里每个元素选定的部分

第一个参数传入，要保留的起始位置的索引；第二个参数传入，要保留的结束位置的下一索引

In [16]:
scores.str.slice(0, 1)

小明    9
小红    6
小杰    7
小丽    8
小华    7
dtype: object